In [1]:
import time
import datetime
import requests
import csv
from tqdm import notebook

In [2]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [3]:
error_url = []

def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [4]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

for user in not_a_user_or_no_data:
    all_username.remove(user)

In [5]:
all_username = [x.strip() for x in all_username]
first_half = all_username[:4999]
second_half = all_username[4999:]

In [6]:
def CollectUserStarredData(writer, username, c):

    user_alias = "usr"+str(c)

    fetched_starred_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username+"/starred?per_page=100&sort=updated")

    for starred in fetched_starred_data:
        try:
            full_name = Select("full_name", starred)
            html_url = Select("html_url", starred)
            description = Select("description", starred)
            language = Select("language", starred)
            topics = Select("topics", starred)
            stargazers_count = Select("stargazers_count", starred)
            writer.writerow([user_alias, full_name, html_url, description, language, topics, stargazers_count])
        except:
            writer.writerow([user_alias, "None", "None", "None", "None", "None", "None"])

In [7]:
print("start time: ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

start time:  21 : 0


In [8]:
with open('all_user_starred_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "full_name", "html_url", "description", "language", "topics", "stargazers_count"])

    c = 0
    for username in notebook.tqdm(first_half):
        try:
            CollectUserStarredData(writer, username, c)
        except:
            pass
        c+=1

    print("first half complete!")

    print("wait for request limit")
    wait_time = 60 - datetime.datetime.now().minute
    print("wait_time = ", wait_time+1)
    for n in notebook.tqdm(range(wait_time+1)):
        time.sleep(60)
    print("request limit = 4999")
    
    for username in notebook.tqdm(second_half):
        try:
            CollectUserStarredData(writer, username, c)
        except:
            pass
        c+=1

print("second half complete!")

  0%|          | 0/4999 [00:00<?, ?it/s]

first half complete!
wait for request limit
wait_time =  8


  0%|          | 0/8 [00:00<?, ?it/s]

request limit = 4999


  0%|          | 0/3584 [00:00<?, ?it/s]

second half complete!


In [9]:
print("stop time: ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

stop time:  22 : 39
